In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

# Create a chat model instance
llm = ChatOpenAI(
   temperature=0.1,
#    streaming=True,
#    callbacks=[StreamingStdOutCallbackHandler()],
)

loader = UnstructuredFileLoader("./files/묵향일부분.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
    # length_function=len,
)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir,)


docs = loader.load_and_split(text_splitter=splitter)

vector_store = FAISS.from_documents(docs, cached_embeddings)

retirever = vector_store.as_retriever()

map_doc_prompt = ChatPromptTemplate([
            (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant 
            to answer the question. 
            Return any relevant text verbatim. 
            If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
])

map_doc_chain = map_doc_prompt | llm


# def map_docs(inputs):
#     # print(inputs) >> document들을 확인 할 수 있다.
#     documents = inputs["documents"]
#     question = inputs["question"]
#     results = []

#     for document in documents:
#         result = map_doc_chain.invoke({"context": document.page_content, "question": question}).content 
#         results.append(result)
#         # map_doc_chain.invoke()가 AI메세지를 반환하기 때문에 그 안에 content만 가져오기위해 .content를 붙임
#     # print(results) >> document의 갯수만큼의 AI 메세지문자열을 볼 수 있다.
#     results = "\n\n".join(results) # 하나의 무자열로 만들되, 줄 바꿈으로 대답을 구분.
#     return 'hello'

# 개선된 map_docs 함수
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]

    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": document.page_content, "question": question}
        ).content 
        for document in documents
    )

map_chain = {"documents": retirever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate([
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
])

chain = {"context": map_chain ,"question":RunnablePassthrough()} | final_prompt | llm

chain.invoke("언급된 기사단의 이름과 단장이름을 알려줘.")




AIMessage(content='아르곤의 성기사단의 단장 이름은 제임스입니다.', additional_kwargs={}, response_metadata={'token_usage': <OpenAIObject at 0x197bd901a30> JSON: {
  "prompt_tokens": 162,
  "completion_tokens": 24,
  "total_tokens": 186,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-be78d8c5-0613-4f91-8bf7-284ed12c9a28-0')